In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing,
import tensorflow as tf
from keras.preprocessing.image import load_img,img_to_array,array_to_img
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # initialize neural network library
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,BatchNormalization,Activation
from keras.optimizers import RMSprop,Adam,SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from tensorflow import keras
import os
from keras.utils.np_utils import to_categorical 
from keras.models import Model
import cv2
from tensorflow import Tensor
from keras.layers import Input, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense
from keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

from keras.applications import ResNet101,InceptionResNetV2,NASNetLarge,ResNet152,ResNet152V2,Xception,MobileNetV2


from sklearn.utils import shuffle

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
label=pd.read_csv("../input/aptos-and-eyepacs/train.csv")
label["path"]=label["id_code"].map(lambda x: os.path.join("../input/aptos-and-eyepacs/APTOS AND EYEPACS/APTOS AND EYEPACS", "{}.png".format(x))) # paths of images

label2=pd.read_csv("../input/aptos2019-blindness-detection/train.csv")
label2["path"]=label2["id_code"].map(lambda x: os.path.join("../input/aptos2019-blindness-detection/train_images", "{}.png".format(x))) # paths of images

#balanced_lab= label.groupby(['diagnosis']).apply(lambda x: x.sample(500, replace = True)).sort_index() 
balanced_lab= label

zeros=balanced_lab[balanced_lab["diagnosis"]==0][:1500]
#zeros_for_test=balanced_lab[balanced_lab["diagnosis"]==0][470:500]
zeros.reset_index(drop=True, inplace=True)
#zeros_for_test.reset_index(drop=True, inplace=True)


ones=balanced_lab[balanced_lab["diagnosis"]==1][:500]
#ones_for_test=balanced_lab[balanced_lab["diagnosis"]==1][340:370]
ones.reset_index(drop=True, inplace=True)
#ones_for_test.reset_index(drop=True, inplace=True)

two=balanced_lab[balanced_lab["diagnosis"]==2][:500]
#two_for_test=balanced_lab[balanced_lab["diagnosis"]==2][470:500]
two.reset_index(drop=True, inplace=True)
#two_for_test.reset_index(drop=True, inplace=True)

tree=balanced_lab[balanced_lab["diagnosis"]==3][:300]
#tree_for_test=balanced_lab[balanced_lab["diagnosis"]==3][173:193]
tree.reset_index(drop=True, inplace=True)
#tree_for_test.reset_index(drop=True, inplace=True)

four=label2[label2["diagnosis"]==4]
#four_for_test=balanced_lab[balanced_lab["diagnosis"]==4][265:295]
four.reset_index(drop=True, inplace=True)
#four_for_test.reset_index(drop=True, inplace=True)

frames=[zeros,ones,two,tree,four]
rebalanced_lab=pd.concat(frames)

#frames_for_test=[zeros_for_test,ones_for_test,two_for_test,tree_for_test,four_for_test]
#rebalanced_lab_for_test=pd.concat(frames_for_test)

#levels_for_test=rebalanced_lab_for_test["diagnosis"]
#levels_for_test=to_categorical(levels_for_test,num_classes=5)

rebalanced_lab = shuffle(rebalanced_lab)

levels=rebalanced_lab["diagnosis"]
levels=to_categorical(levels,num_classes=5)

In [ ]:
rebalanced_lab.head(10)

In [ ]:
rebalanced_lab["diagnosis"].value_counts()

In [ ]:
img_rows, img_cols = 256,256
immatrix=[]
for name in rebalanced_lab["path"]:
    img=cv2.imread(name)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    dim=(img_rows, img_cols)
    resized=cv2.resize(img,dim)
    
    gray=cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY) 
    thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)[1]
    
    # Find contour and sort by contour area
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    
    # Find bounding box and extract ROI
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        ROI = resized[y:y+h, x:x+w]
        break
    crop_son=cv2.resize(ROI,dim)
    
    green_channel = crop_son[:,:,1]   

    green_copy=green_channel.copy()
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clh = clahe.apply(green_copy)
    
    crop_son[:,:,1] = clh   
    
    arka_plan = cv2.medianBlur(crop_son, 59)
    
    maske = cv2.addWeighted(crop_son,1,arka_plan,-1,255)
    
    son_img = cv2.bitwise_and(maske,crop_son)

    immatrix.append(son_img)

In [ ]:
plt.figure()
plt.imshow(resized)

plt.figure()
plt.imshow(thresh)

plt.figure()
plt.imshow(crop_son)

plt.figure()
plt.imshow(arka_plan)

plt.figure()
plt.imshow(maske)

plt.figure()
plt.imshow(son_img)


In [ ]:
# Normalizasyon işlemi
immatrix=np.array(immatrix)   
immatrix=immatrix/255 #normalization for grey form

In [ ]:
# 3 boyutlu olan görsellerimizi 4 boyut'a çevirme. Sebebi sistemin öyle istemesi
immatrix=immatrix.reshape(-1,img_rows,img_cols,3) #1 for grey grey 3 for rgb

In [ ]:
#Train test spliting
x_train,x_val,y_train,y_val=train_test_split(immatrix,levels,random_state=42,test_size=0.1)

immatrix=None
immatrix_for_test=None
levels=None
levels_for_test=None

In [ ]:
datagen = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
        rotation_range=360
        )
      
rn152=ResNet152(weights="imagenet",include_top=False, input_shape=(img_rows, img_cols,3))    


datagen.fit(x_train)

checkopointer=tf.keras.callbacks.ModelCheckpoint(
    filepath="./weihts.h5",
    monitor="val_accuracy",
    verbose=1,
    save_best_only=True,
    save_freq="epoch"
)

model=Sequential()
model.add(rn152)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(256,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.0005),metrics=['accuracy'])

model.summary()

batch_size=25
epochs=200
history=model.fit(datagen.flow(x_train, y_train, 
                               batch_size=batch_size),
                               steps_per_epoch=len(x_train) / batch_size, 
                               epochs=epochs,
                               validation_data=(x_val, y_val),
                               callbacks=[checkopointer])

In [ ]:
max(history.history['val_accuracy'])

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
print(max((history.history['val_accuracy'])))

In [ ]:
#%%Corrolation table

loaded_model = tf.keras.models.load_model('./weihts.h5')

# confusion matrix
import seaborn as sns
# Predict the values from the validation dataset
Y_pred = loaded_model.predict(x_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()